Credit: https://machinelearningmastery.com/text-generation-lstm-recurrent-neural-networks-python-keras/

In [29]:
import numpy
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.callbacks import ModelCheckpoint
from keras.utils import np_utils
import sys

In [8]:
# load ascii text and covert to lowercase
filename = "../../../dataset/Text_data/wonderland.txt"
raw_text = open(filename).read()
raw_text = raw_text.lower()


In [9]:
# create mapping of unique chars to integers
chars = sorted(list(set(raw_text)))
char_to_int = dict((c, i) for i, c in enumerate(chars))

In [13]:
n_chars = len(raw_text)
n_vocab = len(chars)
print ("Total Characters: ", n_chars)
print ("Total Vocab: ", n_vocab)

Total Characters:  144409
Total Vocab:  45


In [16]:
# prepare the dataset of input to output pairs encoded as integers
seq_length = 100
dataX = []
dataY = []
for i in range(0, n_chars - seq_length, 1):
	seq_in = raw_text[i:i + seq_length]
	seq_out = raw_text[i + seq_length]
	dataX.append([char_to_int[char] for char in seq_in])
	dataY.append(char_to_int[seq_out])
n_patterns = len(dataX)
print ("Total Patterns: ", n_patterns)

Total Patterns:  144309


In [17]:
# reshape X to be [samples, time steps, features]
X = numpy.reshape(dataX, (n_patterns, seq_length, 1))
# normalize
X = X / float(n_vocab)
# one hot encode the output variable
y = np_utils.to_categorical(dataY)

In [18]:
# define the LSTM model
model = Sequential()
model.add(LSTM(256, input_shape=(X.shape[1], X.shape[2])))
model.add(Dropout(0.2))
model.add(Dense(y.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam')

In [19]:
# define the checkpoint
filepath="weights-improvement-{epoch:02d}-{loss:.4f}.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=1, save_best_only=True, mode='min')
callbacks_list = [checkpoint]

In [23]:
checkpoint.epochs_since_last_save

0

In [24]:
model.fit(X, y, epochs=20, batch_size=128, callbacks=callbacks_list)

Epoch 1/20
144309/144309 [==============================] - 155s 1ms/step - loss: 2.9725

Epoch 00001: loss improved from inf to 2.97250, saving model to weights-improvement-01-2.9725.hdf5
Epoch 2/20
144309/144309 [==============================] - 154s 1ms/step - loss: 2.7677

Epoch 00002: loss improved from 2.97250 to 2.76775, saving model to weights-improvement-02-2.7677.hdf5
Epoch 3/20
144309/144309 [==============================] - 154s 1ms/step - loss: 2.6647

Epoch 00003: loss improved from 2.76775 to 2.66470, saving model to weights-improvement-03-2.6647.hdf5
Epoch 4/20
144309/144309 [==============================] - 155s 1ms/step - loss: 2.5885

Epoch 00004: loss improved from 2.66470 to 2.58851, saving model to weights-improvement-04-2.5885.hdf5
Epoch 5/20
144309/144309 [==============================] - 155s 1ms/step - loss: 2.5238

Epoch 00005: loss improved from 2.58851 to 2.52384, saving model to weights-improvement-05-2.5238.hdf5
Epoch 6/20
144309/144309 [=============

In [25]:
# load the network weights
filename = "weights-improvement-20-1.9223.hdf5"
model.load_weights(filename)
model.compile(loss='categorical_crossentropy', optimizer='adam')

In [26]:
int_to_char = dict((i, c) for i, c in enumerate(chars))

In [30]:
# pick a random seed
start = numpy.random.randint(0, len(dataX)-1)
pattern = dataX[start]
print ("Seed:")
print ("\"", ''.join([int_to_char[value] for value in pattern]), "\"")
# generate characters
for i in range(1000):
	x = numpy.reshape(pattern, (1, len(pattern), 1))
	x = x / float(n_vocab)
	prediction = model.predict(x, verbose=0)
	index = numpy.argmax(prediction)
	result = int_to_char[index]
	seq_in = [int_to_char[value] for value in pattern]
	sys.stdout.write(result)
	pattern.append(index)
	pattern = pattern[1:len(pattern)]
print ("\nDone.")

Seed:
"  she had gone through that day.

'a likely story indeed!' said the pigeon in a tone of the deepest
c "
areen. ''d'ded tou wail to toe to teln to toal ' she said to herself, 'and that so see th the tou oo aalit it '

'i sh soe the sagt tfree tfree tarter,' said the daterpillar.

'ien a cen,' said the daterpillar.

'ien a cen,' said the daterpillar.

'ien a cen,' said the daterpillar.

'ien a cen,' said the daterpillar.

'ien a cen,' said the daterpillar.

'ien a cen,' said the daterpillar.

'ien a cen,' said the daterpillar.

'ien a cen,' said the daterpillar.

'ien a cen,' said the daterpillar.

'ien a cen,' said the daterpillar.

'ien a cen,' said the daterpillar.

'ien a cen,' said the daterpillar.

'ien a cen,' said the daterpillar.

'ien a cen,' said the daterpillar.

'ien a cen,' said the daterpillar.

'ien a cen,' said the daterpillar.

'ien a cen,' said the daterpillar.

'ien a cen,' said the daterpillar.

'ien a cen,' said the daterpillar.

'ien a cen,' said the daterpi

In [32]:
model2 = Sequential()
model2.add(LSTM(256, input_shape=(X.shape[1], X.shape[2]), return_sequences=True))
model2.add(Dropout(0.2))
model2.add(LSTM(256))
model2.add(Dropout(0.2))
model2.add(Dense(y.shape[1], activation='softmax'))
model2.compile(loss='categorical_crossentropy', optimizer='adam')
# define the checkpoint
filepath="weights-improvement-{epoch:02d}-{loss:.4f}-bigger.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=1, save_best_only=True, mode='min')
callbacks_list = [checkpoint]
# fit the model
model2.fit(X, y, epochs=50, batch_size=64, callbacks=callbacks_list)

Epoch 1/50
144309/144309 [==============================] - 627s 4ms/step - loss: 2.7750

Epoch 00001: loss improved from inf to 2.77500, saving model to weights-improvement-01-2.7750-bigger.hdf5
Epoch 2/50
144309/144309 [==============================] - 631s 4ms/step - loss: 2.4106

Epoch 00002: loss improved from 2.77500 to 2.41058, saving model to weights-improvement-02-2.4106-bigger.hdf5
Epoch 3/50
144309/144309 [==============================] - 627s 4ms/step - loss: 2.2042

Epoch 00003: loss improved from 2.41058 to 2.20421, saving model to weights-improvement-03-2.2042-bigger.hdf5
Epoch 4/50
144309/144309 [==============================] - 625s 4ms/step - loss: 2.0739

Epoch 00004: loss improved from 2.20421 to 2.07395, saving model to weights-improvement-04-2.0739-bigger.hdf5
Epoch 5/50
144309/144309 [==============================] - 625s 4ms/step - loss: 1.9765

Epoch 00005: loss improved from 2.07395 to 1.97651, saving model to weights-improvement-05-1.9765-bigger.hdf5
Epoc

144309/144309 [==============================] - 628s 4ms/step - loss: 1.2787

Epoch 00042: loss did not improve from 1.27777
Epoch 43/50
144309/144309 [==============================] - 628s 4ms/step - loss: 1.2744

Epoch 00043: loss improved from 1.27777 to 1.27443, saving model to weights-improvement-43-1.2744-bigger.hdf5
Epoch 44/50
144309/144309 [==============================] - 628s 4ms/step - loss: 1.2726

Epoch 00044: loss improved from 1.27443 to 1.27257, saving model to weights-improvement-44-1.2726-bigger.hdf5
Epoch 45/50
144309/144309 [==============================] - 627s 4ms/step - loss: 1.2699

Epoch 00045: loss improved from 1.27257 to 1.26987, saving model to weights-improvement-45-1.2699-bigger.hdf5
Epoch 46/50
144309/144309 [==============================] - 628s 4ms/step - loss: 1.2665

Epoch 00046: loss improved from 1.26987 to 1.26647, saving model to weights-improvement-46-1.2665-bigger.hdf5
Epoch 47/50
144309/144309 [==============================] - 627s 4ms/

In [37]:
# load the network weights
filename = "weights-improvement-49-1.2591-bigger.hdf5"
model2.load_weights(filename)
model2.compile(loss='categorical_crossentropy', optimizer='adam')
# pick a random seed
start = numpy.random.randint(0, len(dataX)-1)
pattern = dataX[start]
print ("Seed:")
print ("\"", ''.join([int_to_char[value] for value in pattern]), "\"")
# generate characters
for i in range(1000):
	x = numpy.reshape(pattern, (1, len(pattern), 1))
	x = x / float(n_vocab)
	prediction = model2.predict(x, verbose=0)
	index = numpy.argmax(prediction)
	result = int_to_char[index]
	seq_in = [int_to_char[value] for value in pattern]
	sys.stdout.write(result)
	pattern.append(index)
	pattern = pattern[1:len(pattern)]
print ("\nDone.")

Seed:
"  queen, who had meanwhile been examining the roses.
'off with their heads!' and the procession moved "
 on she was so tale the rame as the way that she was salking in her face, and she was so lane out that she was sealiy and stickly and stre the shmught that she was now the sight size to sist the way that she was seady to sake the look of the thoe, and the thing said to herself, and the three gardeners the sabbit aw the same thing so hear the way that she was now the window, and the thing said that she was now the right size to sist the way that she was salking in her face, and she was so lane out that she was sealiy and stickly and stre the shmught that she was now the sight size to sist the way that she was seady to sake the look of the thoe, and the thing said to herself, and the three gardeners the sabbit aw the same thing so hear the way that she was now the window, and the thing said that she was now the right size to sist the way that she was salking in her face, and she

Seed:
"  of expecting nothing but out-of-the-way
things to happen, that it seemed quite dull and stupid for  "
a little before she had not going on stettion, and then a little shriek so the trial's ang rried to see if she was this time the way that she was bbout in the lock turtle and alice said to the karter with the door, she was now the soog and alice so get out of the wood, 'i've sried the baby?'

'well, i've tried to be a git head beautifyl toop-' said alice. 
'well, i mear the duece,' said the duchess, 'and they live the same thing is to be a good deal the things aeautifu oreer to some tomething is the way to say it a long way in the soo of the soo of her head to her to tp the way that she was now the right size, but the three gardeners the sabbit and the lock turtle sabbit all the tame thing about ier face on the wood, 'i've sried the baby?'

'well, i've tried to be a linttes ' said the mock turtle.

'what a pight to be a dancy ' said the mock turtle. 
'what said the sea,' the karge raid: 'it said in the soog of the shate and the becense of the soog, 
the dormouse said to the kury ae afa
Done